In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from deep_macrofin import PDEModel
from deep_macrofin import ActivationType, OptimizerType, SamplingMethod, plot_loss_df, set_seeds

In [ ]:
latex_var_mapping = {
    # variables
    r"\chi_t": "chi",
    r"\psi_t": "psi",
    "q_t": "q",
    "r_t": "r",
    r"\sigma_t^{q,k}": "sigqk",
    r"\sigma_t^{q,a}": "sigqa",
    r"\iota": "iota",
    r"\Phi": "Phi",
    r"\hat{C}_{e,t}": "ce",
    r"\hat{C}_{h,t}": "ch",
    r"\mu_{ae,t}": "muae",
    r"\sigma_{ae,t}": "sigae",
    r"\sigma_t^{q,k,1}": "sigqk1",
    r"\sigma_t^{z,k}": "sigzk",
    r"\sigma_t^{z,a}": "sigza",
    r"\sigma_{e,t}^{J,k}": "sigejk",
    r"\sigma_{h,t}^{J,k}": "sighjk",
    r"\sigma_{e,t}^{J,a}": "sigeja",
    r"\sigma_{h,t}^{J,a}": "sighja",
    r"\zeta_{e,t}^k": "zetaek",
    r"\zeta_{e,t}^a": "zetaea",
    r"\zeta_{h,t}^k": "zetahk",
    r"\zeta_{h,t}^a": "zetaha",
    r"\epsilon_{e,t}": "epse",
    r"\epsilon_{h,t}": "epsh",
    r"\zeta_{e,t}^1": "zetae1",
    r"\zeta_{e,t}^2": "zetae2",
    r"\zeta_{h,t}^1": "zetah1",
    r"\zeta_{h,t}^2": "zetah2",
    r"\mu_t^z": "muz",
    r"\mu_t^q": "muq",
    r"\mu_{e,t}^J": "muJe",
    r"\mu_{h,t}^J": "muJh",
    r"\mu_{e,t}^R": "muRe",
    r"\mu_{h,t}^R": "muRh",

    # agents
    "J_e": "Je",
    "J_h": "Jh",

    # constants
    r"a_e": "ae",
    r"a_h": "ah",
    r"\sigma": "sigma",
    r"\delta": "delta",
    r"\phi": "phi",
    r"\gamma": "gamma",
    r"\rho": "rho",
    r"\kappa": "kappa",
    r"\lambda_d": "lambdad",
    r"\bar{z}": "zbar",
}

In [ ]:
set_seeds(0)
pde_model = PDEModel("crysis_dynamics", 
                     {"batch_size": 50, "num_epochs": 1000, "loss_log_interval": 100, 
                      "optimizer_type": OptimizerType.Adam, "sampling_method": SamplingMethod.FixedGrid,
                      "loss_balancing": False, "bernoulli_prob": 0.9, "loss_balancing_temp": 0.1, "loss_balancing_alpha": 0.9}, 
                     latex_var_mapping)
pde_model.set_state(["z", "ae"], {"z": [0.05, 0.95], "ae": [0.1, 0.2]})
pde_model.add_agents(["Je", "Jh"], 
                     {"Je": {
                         "positive": True
                        }, 
                      "Jh": {
                         "positive": True
                        }
                     })
pde_model.add_endogs(["chi", "psi", "q", "r", "sigqk", "sigqa"], 
                     {
                        "psi": {
                         "positive": True, 
                        },
                        "q": {
                         "positive": True, 
                        },
                     })
pde_model.add_params({
    "ah": 0.03,
    "sigma": 0.1,
    "delta": 0.05,
    "p": 0.01, 
    "phi": 0.5,
    "gamma": 5,
    "rho": torch.tensor(0.05, device=pde_model.device),
    "kappa": 5,
    "lambdad": 0.03,
    "zbar": 0.1,
    "v": 2.5
})
pde_model.add_equation(r"$\iota &= \frac{q_t - 1}{\kappa}$")
pde_model.add_equation(r"$\Phi &= \frac{\log ( \kappa * \iota + 1)}{\kappa}$")
pde_model.add_equation(r"$\hat{C}_{e,t} &= \rho$")
pde_model.add_equation(r"$\mu_{ae,t} &= p * (0.15 - a_e)$")
pde_model.add_equation(r"$\sigma_{ae,t} &= v * (0.2 - a_e) * (a_e - 0.1)$")
pde_model.add_equation(r"$\sigma_t^{q,k,1} &= \sigma + \sigma_t^{q,k}$")
pde_model.add_equation(r"$\sigma_t^{z,k} &= \left(\frac{\chi_t * \psi_t}{z} - 1\right) * \sigma_t^{q,k,1}$")
pde_model.add_equation(r"$\sigma_t^{z,a} &= \left(\frac{\chi_t * \psi_t}{z} - 1\right) * \sigma_t^{q,k,1}$")
pde_model.add_equation(r"$\sigma_{e,t}^{J,k} &= \frac{1}{J_e} * \frac{\partial J_e}{\partial z} * z * \sigma_t^{z,k}$")
pde_model.add_equation(r"$\sigma_{h,t}^{J,k} &= \frac{1}{J_h} * \frac{\partial J_h}{\partial z} * z * \sigma_t^{z,k}$")
pde_model.add_equation(r"$\sigma_{e,t}^{J,a} &= \frac{1}{J_e} * \left(\frac{\partial J_e}{\partial a_e} * \sigma_{ae,t} + \frac{\partial J_e}{\partial z} * z * \sigma_t^{z,a} \right)$")
pde_model.add_equation(r"$\sigma_{h,t}^{J,a} &= \frac{1}{J_h} * \left(\frac{\partial J_h}{\partial a_e} * \sigma_{ae,t} + \frac{\partial J_h}{\partial z} * z * \sigma_t^{z,a} \right)$")
pde_model.add_equation(r"$\zeta_{e,t}^k &= - (1-\gamma) * \sigma_{e,t}^{J,k} + \sigma_t^{z,k} + \sigma_t^{q,k} + \gamma * \sigma$")
pde_model.add_equation(r"$\zeta_{e,t}^a &= - (1-\gamma) * \sigma_{e,t}^{J,a} + \sigma_t^{z,a} + \sigma_t^{q,a}$")
pde_model.add_equation(r"$\zeta_{h,t}^k &= - (1-\gamma) * \sigma_{h,t}^{J,k} - \frac{z}{1-z} * \sigma_t^{z,k} + \sigma_t^{q,k} + \gamma * \sigma $")
pde_model.add_equation(r"$\zeta_{h,t}^a &= -(1-\gamma) * \sigma_{h,t}^{J,a} - \frac{z}{1-z} * \sigma_t^{z,a} + \sigma_t^{q,a}$")
pde_model.add_equation(r"$\epsilon_{e,t} &= \zeta_{e,t}^k * \sigma_t^{q,k,1} + \zeta_{e,t}^a * \sigma_t^{q,a} + \phi * (\zeta_{e,t}^a * \sigma_t^{q,k,1} + \zeta_{e,t}^k * \sigma_t^{q,a})$")
pde_model.add_equation(r"$\epsilon_{h,t} &= \zeta_{h,t}^k * \sigma_t^{q,k,1} + \zeta_{h,t}^a * \sigma_t^{q,a} + \phi * (\zeta_{h,t}^a * \sigma_t^{q,k,1} + \zeta_{h,t}^k * \sigma_t^{q,a})$")
pde_model.add_equation(r"$\zeta_{e,t}^1 &= \zeta_{e,t}^k + \phi * \zeta_{e,t}^a$")
pde_model.add_equation(r"$\zeta_{h,t}^1 &= \zeta_{h,t}^k + \phi * \zeta_{h,t}^a$")
pde_model.add_equation(r"$\zeta_{e,t}^2 &= \zeta_{e,t}^a + \phi * \zeta_{e,t}^k$")
pde_model.add_equation(r"$\zeta_{h,t}^2 &= \zeta_{h,t}^a + \phi * \zeta_{h,t}^k$")
pde_model.add_equation(r"$\mu_t^z &= \frac{a_e - \iota}{q_t} - \hat{C}_{e,t} + \left(\frac{\chi_t * \psi_t}{z} - 1\right) * (\sigma_t^{q,k,1} * (\zeta_{e,t}^1 - \sigma_t^{q,k,1}) + \sigma_t^{q,a} * (\zeta_{e,t}^2 - \sigma_t^{q,a}) - 2 * \phi * \sigma_t^{q,k,1} * \sigma_t^{q,a}) + (1-\chi_t) * (\sigma_t^{q,k,1} * (\zeta_{e,t}^1 - \zeta_{h,t}^1) + \sigma_t^{q,a} * (\zeta_{e,t}^2 - \zeta_{h,t}^2)) + \frac{\lambda_d}{z} * (\bar{z} - z)$")
pde_model.add_equation(r"$\mu_t^q &= \frac{1}{q_t} * \left( \frac{\partial q_t}{\partial z} * \mu_t^z + \frac{\partial q_t}{\partial a_e} * \mu_{ae,t} + \frac{1}{2} * \frac{\partial^2 q_t}{\partial z^2} * ((\sigma_t^{z,k})^2 + (\sigma_t^{z,a})^2 + 2 * \phi * \sigma_t^{z,k} * \sigma_t^{z,a})\right) + \frac{1}{q_t} * \left( \frac{1}{2} * \frac{\partial^2 q_t}{\partial a_e^2} * \sigma_{ae,t}^2 + \frac{\partial^2 q_t}{\partial z \partial a_e} * (\phi * \sigma_t^{z,k} * \sigma_{ae,t} + \sigma_t^{z,a} * \sigma_{ae,t})\right)$")
pde_model.add_equation(r"$\mu_{e,t}^J &= \frac{1}{J_e} * \left(\frac{\partial J_e}{\partial z} * \mu_t^z * z + \frac{\partial J_e}{\partial a_e} * \mu_{ae,t} + \frac{1}{2} * \frac{\partial^2  J_e}{\partial z^2} * z^2 * \left( (\sigma_t^{z,k})^2 + (\sigma_t^{z,a})^2 + 2 * \phi * \sigma_t^{z,k}* \sigma_t^{z,a} \right) \right) + \frac{1}{J_e} * \left(\frac{1}{2} * \frac{\partial^2 J_e}{\partial a_e^2} * \sigma_{ae,t}^2 + \frac{\partial^2 J_e}{\partial z \partial a_e} * (\phi * \sigma_t^{z,k} + \sigma_t^{z,a}) * \sigma_{ae,t} * z \right)$")
pde_model.add_equation(r"$\mu_{h,t}^J &= \frac{1}{J_h} * \left(\frac{\partial J_h}{\partial z} * \mu_t^z * z + \frac{\partial J_h}{\partial a_e} * \mu_{ae,t} + \frac{1}{2} * \frac{\partial^2  J_h}{\partial z^2} * z^2 * \left( (\sigma_t^{z,k})^2 + (\sigma_t^{z,a})^2 + 2 * \phi * \sigma_t^{z,k}* \sigma_t^{z,a} \right) \right) + \frac{1}{J_h} * \left(\frac{1}{2} * \frac{\partial^2 J_h}{\partial a_e^2} * \sigma_{ae,t}^2 + \frac{\partial^2 J_h}{\partial z \partial a_e} * (\phi * \sigma_t^{z,k} + \sigma_t^{z,a}) * \sigma_{ae,t} * z \right)$")
pde_model.add_equation(r"$\mu_{e,t}^R &= \mu_t^q + \Phi - \delta + \sigma * \sigma_t^{q,k} + \phi * \sigma * \sigma_t^{q,a} + \frac{a_e - \iota}{q_t}$")
pde_model.add_equation(r"$\mu_{h,t}^R &= \mu_t^q + \Phi - \delta + \sigma * \sigma_t^{q,k} + \phi * \sigma * \sigma_t^{q,a} + \frac{a_h - \iota}{q_t}$")

pde_model.add_endog_equation(r"$\sigma_t^{q,k} &= \frac{1}{q_t} * \frac{\partial q_t}{\partial z} * \sigma_t^{z,k}$")
pde_model.add_endog_equation(r"$\sigma_t^{q,a} &= \frac{1}{q_t} * \left(\frac{\partial q_t}{\partial z} * \sigma_t^{z,a} + \frac{\partial q_t}{\partial a_e} * \sigma_{ae,t} \right)$")
pde_model.add_endog_equation(r"$\rho * q_t &= \psi_t * (a_e - \iota) + (1-\psi_t) * (a_h - \iota)$")
pde_model.add_endog_equation(r"$\chi_t * \epsilon_{e,t} + (1-\chi_t) * \epsilon_{h,t} &= \frac{a_e - \iota}{q_t} + \Phi - \delta + \mu_t^q + \sigma * \sigma_t^{q,k} + \phi * \sigma * \sigma_t^{q,a} - r_t$")

pde_model.add_hjb_equation(r"$\rho * (\log(\rho) - \log(J_e) + \log(q_t * z)) + \Phi - \delta - \frac{\gamma}{2} * \sigma^2 - \frac{\gamma}{2} * ((\sigma_{e,t}^{J,k})^2 + (\sigma_{e,t}^{J,a})^2 + 2 * \phi * \sigma_{e,t}^{J,k} * \sigma_{e,t}^{J,a}) + \mu_{e,t}^J + (1-\gamma) * (\sigma * \sigma_{e,t}^{J,k} + \phi * \sigma * \sigma_{e,t}^{J,a})$")
pde_model.add_hjb_equation(r"$\rho * (\log(\rho) - \log(J_h) + \log(q_t * (1-z))) + \Phi - \delta - \frac{\gamma}{2} * \sigma^2 - \frac{\gamma}{2} * ((\sigma_{h,t}^{J,k})^2 + (\sigma_{h,t}^{J,a})^2 + 2 * \phi * \sigma_{h,t}^{J,k} * \sigma_{h,t}^{J,a}) + \mu_{h,t}^J + (1-\gamma) * (\sigma * \sigma_{h,t}^{J,k} + \phi * \sigma * \sigma_{h,t}^{J,a})$")
print(pde_model)
if not os.path.exists("./models/crysis_dynamics1/model.pt"):
    pde_model.train_model("./models/crysis_dynamics1", "model.pt", True)
    pde_model.load_model(torch.load("./models/crysis_dynamics1/model_best.pt"))
    pde_model.eval_model(True)
else:
    pde_model.load_model(torch.load("./models/crysis_dynamics1/model_best.pt"))
    pde_model.eval_model(True)

In [ ]:
plot_loss_df(fn="./models/crysis_dynamics1/model_min_loss.csv", loss_plot_fn="./models/crysis_dynamics1/model_min_loss.png")
# plot_loss_df(fn="./models/crysis_dynamics1/model_loss_weight.csv", loss_plot_fn="./models/crysis_dynamics1/model_loss_weight.png", log_loss=False)
pde_model.plot_vars([r"$q_t$", r"$\mu_t^z$", # r"$\sigma+\sigma_q = \sigma_t^{q,k,1}$", 
                     r"$\|\sigma_t^R\|^2 = (\sigma_t^{q,k,1})^2 + (\sigma_t^{q,a})^2$",
                     r"$\psi_t$",
                     r"$\|\sigma_t^z\|^2 = (\sigma_t^{z,k})^2 + (\sigma_t^{z,a})^2$", 
                     r"$rp = \mu_{e,t}^R - r_t$"], ncols=3)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(18, 12))
for x_val, linestyle in [(0.1, "-"), (0.15, ":"), (0.2, "--")]:
    sv = torch.ones((100, 2), device=pde_model.device) * x_val
    sv[:, 0] = torch.linspace(0.05, 0.95, 100)
    for i, sv_name in enumerate(pde_model.state_variables):
        pde_model.variable_val_dict[sv_name] = sv[:, i:i+1]
    pde_model.update_variables(sv)
    price = pde_model.variable_val_dict["q"]
    psi = pde_model.variable_val_dict["psi"]
    muz = pde_model.variable_val_dict["muz"]
    ret_vol = pde_model.variable_val_dict["sigqk1"] ** 2 + pde_model.variable_val_dict["sigqa"] ** 2
    diffusion_z = pde_model.variable_val_dict["sigzk"] ** 2 + pde_model.variable_val_dict["sigza"] ** 2
    rp = pde_model.variable_val_dict["muRe"] - pde_model.variable_val_dict["r"]
    ax[0][0].plot(sv[:, 0].detach().cpu().numpy(), price.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")
    ax[0][1].plot(sv[:, 0].detach().cpu().numpy(), muz.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")
    ax[0][2].plot(sv[:, 0].detach().cpu().numpy(), ret_vol.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")

    ax[1][0].plot(sv[:, 0].detach().cpu().numpy(), psi.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")
    ax[1][1].plot(sv[:, 0].detach().cpu().numpy(), diffusion_z.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")
    ax[1][2].plot(sv[:, 0].detach().cpu().numpy(), rp.detach().cpu().numpy().reshape(-1), linestyle=linestyle, label=f"x={x_val}")

for i, (title, ylabel) in enumerate([("Price", r"$q_t$"), ("Drift of wealth share", r"$\mu_t^z$"), 
                                     ("Return volatility", r"$\|\sigma_t^R\|^2$"), ("Capital Share: Experts", r"$\psi$"),
                                     ("Diffusion of wealth share", r"$\|\sigma_t^z\|^2$"), ("Risk premium", "$\mu_{e,t}^R - r_t$")]):
    row = i // 3
    col = i % 3
    ax[row][col].set_title(title)
    ax[row][col].set_xlabel("z")
    ax[row][col].set_ylabel(ylabel)
    ax[row][col].legend()